In [1]:
import pandas as pd
import pandas as pd
from pathlib import Path
from config import *
from ppp_prediction.plot.utils import save_fig
import matplotlib.pyplot as plt

[0.0, 0.5, 1.0]
[0.0, 50.0, 100.0]
[0.0, 20.0, 40.0, 60.0, 80.0]


## Load data

In [2]:
need_center = ["AZ-TR", "DX-EV"]

In [3]:
# 2) load predictions and labels
Ummae = pd.read_feather("Raw/Ummae.feather").query("DataSetName in @need_center")
clinical_label = pd.read_feather("Raw/ClinicalLabels.feather").query(
    "DataSetName in @need_center"
)
# mean of prediction of same individuals
Ummae_mean = (
    Ummae.groupby(["eid", "type", "DataSetName"])
    .agg(
        {
            **{col: "mean" for col in Ummae.columns[3:]},
        }
    )
    .reset_index(drop=False)
).merge(clinical_label, on=["eid", "DataSetName"], suffixes=["_Ummae", ""])
Ummae_mean["Sex"] = Ummae_mean["Sex"].map({"Male": 1, "Female": 0})
Ummae_mean.to_feather(RawDataDir / "Ummae_mean.feather")
Ummae_mean

,eid,type,DataSetName,AG_Ummae,ALP_Ummae,ALT_Ummae,APTT_Ummae,AST_Ummae,Alb_Ummae,BMI_Ummae,...,Urea,WBC,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age,Sex,Stenosis
0,00286da6700c9df42ae7b741abf70c1b,test,DX-EV,15.633866,80.345654,17.568888,30.117004,19.861627,43.957889,25.211603,...,3.30,6.31,45.0,NaN,NaN,NaN,NaN,69.0,0,0.0
1,0028d537189c92f3d0aab626c7cfd132,test,DX-EV,15.576301,80.663582,17.522696,30.358721,19.926036,44.624769,24.767671,...,6.50,5.02,55.0,36.91,NaN,NaN,NaN,69.0,0,0.0
2,00299c21cbaafb82dea6deefaffbd3b3,train,AZ-TR,16.548534,74.757131,26.263929,31.968572,20.807044,46.680381,25.537003,...,6.60,6.08,64.0,0.46,2.9,2.11,0.35,50.0,1,1.0
3,00411460f7c92d2124a67ea0f4cb5f85,test,AZ-TR,18.058273,75.389764,24.605602,31.838446,20.519264,45.689174,27.452344,...,4.76,6.53,85.0,2.08,73.9,2.63,0.55,54.0,1,1.0
4,00494b8a054a33de725a244dd1fdc9ec,test,AZ-TR,16.422336,74.652830,23.356261,31.781211,20.465901,46.124434,24.232457,...,4.84,5.86,72.0,0.29,0.0,2.07,0.32,56.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5593,ff7f406bc96dc190f93e4c8d7f80498a,train,AZ-TR,17.752752,75.639534,21.653303,31.579250,20.092318,44.853260,23.067019,...,5.80,6.23,56.0,0.59,1.7,2.64,0.47,66.0,1,0.0
5594,ffaf45b4f27a626bd02370a93248527b,test,DX-EV,15.741298,85.416766,18.816056,30.153703,19.952209,45.543306,28.005491,...,5.10,5.90,58.0,17.68,NaN,NaN,NaN,69.0,0,1.0
5595,ffba659242eeda444d68aff43d078a46,validation,AZ-TR,15.567951,79.713266,23.967179,31.638487,20.082053,44.459733,26.779657,...,2.83,6.60,NaN,9.44,24.6,2.16,NaN,60.0,1,1.0
5596,ffc7b0fd6ddbcf7f1521260447f90d9a,train,AZ-TR,16.047764,75.364927,29.592352,32.148088,21.017451,46.964378,29.337831,...,4.04,9.44,91.0,4.95,16.0,2.83,0.36,27.0,1,1.0


In [4]:
clinical_label.columns

Index(['eid', 'DataSetName', 'AG', 'ALP', 'ALT', 'APTT', 'AST', 'Alb', 'BMI',
       'BNP', 'C1q', 'Cl', 'CK', 'CK-MB', 'Cr', 'D-Bil', 'D-Dimer',
       'eGFR(CKD-EPI)', 'FBG', 'FFA', 'FT3', 'FT4', 'GGT', 'Glu', 'HCT',
       'HDLC', 'Hb', 'HbA1C%', 'Hcy', 'Height', 'INR', 'Isn', 'K', 'LDH',
       'LDLC', 'LPa', 'LYM', 'LYM%', 'MCH', 'MCHC', 'MCV', 'MPV', 'NE', 'NE%',
       'Na', 'PCT', 'PLT', 'RBC', 'T-Bil', 'T3', 'T4', 'TC', 'TG', 'TP', 'TSH',
       'UA', 'Urea', 'WBC', 'Weight', 'hs-CRP', 'hsTnI', 'nonHDL', 'sdLDL',
       'Age', 'Sex', 'Stenosis'],
      dtype='object')

## Association Analysis

In [5]:
to_cal_assoc_data = Ummae_mean.query(
    "DataSetName in @need_center and type =='test'"
).copy()
to_cal_assoc_data

,eid,type,DataSetName,AG_Ummae,ALP_Ummae,ALT_Ummae,APTT_Ummae,AST_Ummae,Alb_Ummae,BMI_Ummae,...,Urea,WBC,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age,Sex,Stenosis
0,00286da6700c9df42ae7b741abf70c1b,test,DX-EV,15.633866,80.345654,17.568888,30.117004,19.861627,43.957889,25.211603,...,3.30,6.31,45.0,NaN,NaN,NaN,NaN,69.0,0,0.0
1,0028d537189c92f3d0aab626c7cfd132,test,DX-EV,15.576301,80.663582,17.522696,30.358721,19.926036,44.624769,24.767671,...,6.50,5.02,55.0,36.91,NaN,NaN,NaN,69.0,0,0.0
3,00411460f7c92d2124a67ea0f4cb5f85,test,AZ-TR,18.058273,75.389764,24.605602,31.838446,20.519264,45.689174,27.452344,...,4.76,6.53,85.0,2.08,73.9,2.63,0.55,54.0,1,1.0
4,00494b8a054a33de725a244dd1fdc9ec,test,AZ-TR,16.422336,74.652830,23.356261,31.781211,20.465901,46.124434,24.232457,...,4.84,5.86,72.0,0.29,0.0,2.07,0.32,56.0,1,1.0
14,00ab2592e376cc62a3fd3d82a2497a29,test,AZ-TR,15.420380,80.927822,21.013279,31.482744,20.086543,45.138908,25.942478,...,5.85,6.21,63.0,0.94,12.0,1.79,0.41,69.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5576,fe74cf5d214858f602408c12c7916afe,test,DX-EV,15.560330,81.140633,18.786305,30.904483,20.007380,45.082222,22.647569,...,4.60,4.91,60.0,1.21,NaN,NaN,NaN,63.0,0,1.0
5581,fe9eca08e151c83edf145bc5fee2b41c,test,DX-EV,15.903710,84.778509,17.619776,30.140268,19.892420,44.294562,27.915181,...,5.00,6.24,73.0,NaN,NaN,NaN,NaN,72.0,0,0.0
5583,fea8d4b137323af62aeb0f4b7c97153a,test,DX-EV,15.683733,78.825900,20.449979,30.470933,19.900374,45.425767,25.587461,...,6.20,5.99,70.0,1.51,NaN,NaN,NaN,59.0,1,1.0
5586,ff2b196ac34879baf3d052824dedb0dc,test,AZ-TR,17.551215,79.954737,21.660350,31.476865,20.752600,45.771065,24.466548,...,6.76,NaN,64.0,0.83,4.9,2.24,0.67,68.0,0,1.0


In [6]:
feature_list = [
    "BNP",
    "CK",
    "CK-MB",
    "D-Dimer",
    "HDLC",
    "HbA1C%",
    "Hcy",
    "LDH",
    "LDLC",
    "LPa",
    "TC",
    "TG",
    "hs-CRP",
    "hsTnI",
    "nonHDL",
    "sdLDL",
]

In [7]:
from ppp_prediction.corr_v3 import cal_corr_v3
import pickle
import numpy as np

# remove outlier
for col in feature_list:

    to_cal_assoc_data[col] = np.clip(
        to_cal_assoc_data[col],
        to_cal_assoc_data[col].dropna().min(),
        np.percentile(to_cal_assoc_data[col].dropna(), 95),
    )

In [8]:
# save to_cal_assoc_data
to_cal_assoc_data.reset_index(drop=True).to_feather(
    RawDataDir / "to_cal_assoc_data.feather"
)

In [9]:
data_dict_dir = RawDataDir / "Assoc_data_dict_stenosis.pkl"
if data_dict_dir.exists():
    data_dict = pd.read_pickle(data_dict_dir)
    res_real = data_dict["Real"]
    res_ummae = data_dict["Ummae"]
else:
    res_real = cal_corr_v3(
        df=to_cal_assoc_data,
        x=feature_list,
        y="Stenosis",
        cov=["Age", "Sex", "Height", "Weight", "BMI"],
        norm_x="zscore",
        # cat_cols=cat_cols,
    )
    feature_list_ummae = [f"{tgt}_Ummae" for tgt in feature_list]

    res_ummae = cal_corr_v3(
        df=to_cal_assoc_data,
        x=feature_list_ummae,
        y="Stenosis",
        cov=["Age", "Sex", "Height", "Weight", "BMI"],
        norm_x="zscore",
        # cat_cols=cat_cols,
    )
    data_dict = {"Ummae": res_ummae, "Real": res_real}

    pickle.dump(data_dict, open(data_dict_dir, "wb"))

## Disease Risk Evaluation Model

In [10]:
model_data = Ummae_mean
all_clinical_ummae_traits = [
    "BNP",
    "CK",
    "CK-MB",
    "D-Dimer",
    "HDLC",
    "HbA1C%",
    "Hcy",
    "LDH",
    "LDLC",
    "LPa",
    "TC",
    "TG",
    "hs-CRP",
    "hsTnI",
    "nonHDL",
    "sdLDL",
]
Model = "Ummae"
Disease = "Stenosis"

In [11]:
clinical = [
    "TG",
    "TC",
    "HDLC",
    "LDLC",
]
Clinical_ummae = [f"{trait}_{Model}" for trait in clinical]
Clinical_ummae
Clinical_ummae_full = [f"{trait}_{Model}" for trait in all_clinical_ummae_traits]
basic = ["Age", "Sex", "Height", "Weight"]
Clinical_agesex = clinical + basic
Clinical_ummae_agesex = Clinical_ummae + basic
Clinical_ummae_full_agesex = Clinical_ummae_full + basic

model_data[clinical].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5598 entries, 0 to 5597
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TG      5481 non-null   float64
 1   TC      5481 non-null   float64
 2   HDLC    5478 non-null   float64
 3   LDLC    5379 non-null   float64
dtypes: float64(4)
memory usage: 218.7 KB


In [12]:
from ppp_prediction.model_v2.DiseaseScore_V2 import (
    DiseaseScoreModel_V2,
    config_dict_to_df,
)
from ppp_prediction.model_v2.models import *

model_table = config_dict_to_df(
    {
        "Clinical": {
            "xvar": Clinical_agesex,
            "load_local_cache": True,
            "model": fit_ensemble_model_simple_v2,
            "config": {
                "engine": "sklearn",
                "need_scale": True,
                "random_state": 42,
                "method": "Logistic",
            },
        },
        "Predicted Clinical": {
            "xvar": Clinical_ummae_agesex,
            "model": fit_ensemble_model_simple_v2,
            "load_local_cache": True,
            "config": {
                "engine": "sklearn",
                "need_scale": True,
                "random_state": 42,
                "method": "Logistic",
            },
        },
        "Predicted Clinical Full": {
            "xvar": Clinical_ummae_full_agesex,
            "model": fit_ensemble_model_simple_v2,
            "load_local_cache": True,
            "config": {
                "engine": "sklearn",
                "need_scale": True,
                "random_state": 42,
                "method": "Logistic",
            },
        },
    },
    index_name=("Score"),
)

/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/dask/array/chunk_types.py:110: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------



In [13]:
train_eid = model_data.query("DataSetName == 'AZ-TR'")[["eid"]].reset_index(drop=True)

test_eid = model_data.query("eid not in @train_eid.eid")[["eid"]].reset_index(drop=True)
test_eid

CADModel = DiseaseScoreModel_V2(
    disease_df=model_data.copy(),
    model_table=model_table,
    label=Disease,
    disease_name=Disease,
    # test_size=0.5,
    train_eid=train_eid.eid,
    test_eid=test_eid.eid,
    other_keep_cols=[col for col in model_data.columns if col not in [Disease, "eid"]],
    saveDir="ModelTrain",
)

CADModel.update_model()
CADModel.model_table

INFO:root:Loading data with train cases 3485.0 and test cases 513.0 of Stenosis, while 129 columns
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/model_v2/DiseaseScore_V2.py:726: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/model_v2/DiseaseScore_V2.py:727: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
INFO:root:Drop NA by Stenosis and None and None in train and test and left 4698 and 900 with train cases 3485.0 and test cases 513.0
INFO:root:Load model from ModelTrain/Clinical.pkl of Clinical
INFO:root:Load score from ModelTrain/Clinical_train.feather 

INFO:root:Load model from ModelTrain/Predicted Clinical Full.pkl of Predicted Clinical Full
INFO:root:Load score from ModelTrain/Predicted Clinical Full_train.feather and ModelTrain/Predicted Clinical Full_test.feather


param  \
                                                                      xvar   
Score                                                                        
Clinical                    [TG, TC, HDLC, LDLC, Age, Sex, Height, Weight]   
Predicted Clinical       [TG_Ummae, TC_Ummae, HDLC_Ummae, LDLC_Ummae, A...   
Predicted Clinical Full  [BNP_Ummae, CK_Ummae, CK-MB_Ummae, D-Dimer_Umm...   

                                          \
                        load_local_cache   
Score                                      
Clinical                            True   
Predicted Clinical                  True   
Predicted Clinical Full             True   

                                                                            \
                                                                     model   
Score                                                                        
Clinical                 <function fit_ensemble_model_simple_v2 at 0x70...   
Predicted Clinical       <function fit_ensemble_model_simple_v2 at 0x70...   
Predicted Clinical Full  <function fit_ensemble_model_simple_v2 at 0x70...   

                                                                            \
                                                                    config   
Score                                                                        
Clinical                 {'engine': 'sklearn', 'need_scale': True, 'ran...   
Predicted Clinical       {'engine': 'sklearn', 'need_scale': True, 'ran...   
Predicted Clinical Full  {'engine': 'sklearn', 'need_scale': True, 'ran...   

                              auc                                          \
                              AUC   AUC_UCI   AUC_LCI       ACC   ACC_UCI   
Score                                                                       
Clinical                 0.613717  0.645711  0.571431  0.590327  0.616003   
Predicted Clinical       0.655070  0.697444  0.619585  0.634085  0.667387   
Predicted Clinical Full  0.663180  0.696205  0.627424  0.650376  0.685526   

                                   ...                                      \
                          ACC_LCI  ... Specificity_LCI       APR   APR_UCI   
Score                              ...                                       
Clinical                 0.549004  ...        0.611581  0.675204  0.732477   
Predicted Clinical       0.605263  ...        0.593792  0.668032  0.711085   
Predicted Clinical Full  0.615883  ...        0.521526  0.690949  0.747088   

                                                          status  \
                          APR_LCI      N N_case N_control fitted   
Score                                                              
Clinical                 0.628399  703.0  408.0     295.0    1.0   
Predicted Clinical       0.617550  798.0  450.0     348.0    1.0   
Predicted Clinical Full  0.651830  798.0  450.0     348.0    1.0   

                                                                     model  \
                                                                     model   
Score                                                                        
Clinical                 (StandardScaler(), LogisticRegression(class_we...   
Predicted Clinical       (StandardScaler(), LogisticRegression(class_we...   
Predicted Clinical Full  (StandardScaler(), LogisticRegression(class_we...   

                                           basic  
                                      score_name  
Score                                             
Clinical                                Clinical  
Predicted Clinical            Predicted Clinical  
Predicted Clinical Full  Predicted Clinical Full  

[3 rows x 28 columns]

In [14]:
to_cal_scores = CADModel.model_table["basic", "score_name"].values.tolist()
test_scores = CADModel.test_score
train_scores = CADModel.train_score

# norm test_scores

test_scores[to_cal_scores] = (
    test_scores[to_cal_scores] - train_scores[to_cal_scores].mean()
) / train_scores[to_cal_scores].std()
test_scores = test_scores.dropna(subset=to_cal_scores)
test_scores

,eid,Stenosis,type,DataSetName,AG_Ummae,ALP_Ummae,ALT_Ummae,APTT_Ummae,AST_Ummae,Alb_Ummae,...,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age,Sex,Clinical,Predicted Clinical,Predicted Clinical Full
0,00286da6700c9df42ae7b741abf70c1b,0.0,test,DX-EV,15.633866,80.345654,17.568888,30.117004,19.861627,43.957889,...,45.0,NaN,NaN,NaN,NaN,69.0,0,-1.636073,-1.641001,-0.089500
1,0028d537189c92f3d0aab626c7cfd132,0.0,test,DX-EV,15.576301,80.663582,17.522696,30.358721,19.926036,44.624769,...,55.0,36.91,NaN,NaN,NaN,69.0,0,0.410182,-2.181934,-0.775324
2,00b172dd41ec17e59fc3fbfe3e5ad040,1.0,test,DX-EV,15.599855,85.232132,19.338253,30.669607,20.114129,44.417700,...,65.0,1.27,NaN,NaN,NaN,63.0,0,-1.100143,-2.703307,-1.716836
3,012115a7de561d1a3855dd18d8436d29,1.0,test,DX-EV,15.538857,75.078251,23.139159,31.451798,20.130519,46.481562,...,70.0,0.30,NaN,NaN,NaN,50.0,1,0.228572,-0.433940,1.098653
4,0158e4920ed2c05fc493109c0f212912,1.0,test,DX-EV,15.826842,76.170011,19.734403,30.514102,19.903454,43.827881,...,64.0,2.42,NaN,NaN,NaN,63.0,1,1.200125,0.370132,0.538091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,fe73855720140766aef47f71fe84e24b,1.0,test,DX-EV,15.693959,82.858584,18.030018,31.003819,20.001734,44.437099,...,75.0,NaN,NaN,NaN,NaN,69.0,0,-1.021677,-2.798529,-1.101126
896,fe74cf5d214858f602408c12c7916afe,1.0,test,DX-EV,15.560330,81.140633,18.786305,30.904483,20.007380,45.082222,...,60.0,1.21,NaN,NaN,NaN,63.0,0,-1.348961,-2.790504,-1.173310
897,fe9eca08e151c83edf145bc5fee2b41c,0.0,test,DX-EV,15.903710,84.778509,17.619776,30.140268,19.892420,44.294562,...,73.0,NaN,NaN,NaN,NaN,72.0,0,-0.458183,-1.822918,-1.256662
898,fea8d4b137323af62aeb0f4b7c97153a,1.0,test,DX-EV,15.683733,78.825900,20.449979,30.470933,19.900374,45.425767,...,70.0,1.51,NaN,NaN,NaN,59.0,1,0.881930,0.093886,1.002541


In [15]:
test_scores.reset_index(drop=True).to_feather(
    RawDataDir / "ClinicalModelScores.feather"
)

In [18]:
from ppp_prediction.metrics import cal_binary_metrics

# predicted_result_save_dir = RawDataDir / "DiseasePredicted.feather"

result_list = []
for col in to_cal_scores:
    to_cal_df = test_scores[[col, Disease, "DataSetName"]].dropna()
    score_metrics = to_cal_df.groupby(["DataSetName"]).apply(
        lambda x: pd.Series(
            cal_binary_metrics(
                x[[col, Disease]].dropna()[Disease],
                x[[col, Disease]].dropna()[col],
                ci=True,
                n_resamples=1000,
                # pos_label=1,
            )
        )
    )
    score_metrics["score"] = col

    result_list.append(score_metrics)

result_list_df = pd.concat(result_list)

result_list_df.insert(0, "score", result_list_df.pop("score"))
result_list_df = result_list_df.reset_index(drop=False)
# result_list_df.to_feather(predicted_result_save_dir)

result_list_df

,DataSetName,score,AUC,AUC_UCI,AUC_LCI,ACC,ACC_UCI,ACC_LCI,Macro_F1,Macro_F1_UCI,...,Sensitivity_LCI,Specificity,Specificity_UCI,Specificity_LCI,APR,APR_UCI,APR_LCI,N,N_case,N_control
0,DX-EV,Clinical,0.613717,0.656375,0.570392,0.590327,0.628734,0.556188,0.589808,0.624415,...,0.490963,0.661017,0.716594,0.609648,0.675204,0.728106,0.627826,703.0,408.0,295.0
1,DX-EV,Predicted Clinical,0.649103,0.690248,0.609082,0.634424,0.667141,0.598862,0.631369,0.666462,...,0.576828,0.647458,0.703707,0.593557,0.677566,0.730343,0.626436,703.0,408.0,295.0
2,DX-EV,Predicted Clinical Full,0.656514,0.695673,0.617527,0.648649,0.687055,0.611664,0.637738,0.673434,...,0.666667,0.566102,0.622421,0.509745,0.698493,0.746996,0.646338,703.0,408.0,295.0


In [19]:
result_list_df.to_csv(RawDataDir / "ClinicalModelPerformance.csv", index=False)

## Relative importance of the features

In [16]:
train_data = model_data.query("DataSetName == 'AZ-TR'").copy()

# save dir
RelativeDataDir = RawDataDir / "RelativeImportance_Cardiovascular_full.csv"
RelativeDataDir

PosixPath('V1/paper/RawData/RelativeImportance_Cardiovascular_full.csv')

In [17]:
from utils import *

if not RelativeDataDir.exists():
    # 1) cal relative importance for Ummae predicted values
    omics = Clinical_ummae_full + basic

    RelativeImportanceData = train_data[omics + ["Stenosis"]].copy()
    RelativeImportanceDataFromat = columnsFormatV1(RelativeImportanceData)
    formated_data = RelativeImportanceDataFromat.format(RelativeImportanceData)

    omics_format = [RelativeImportanceDataFromat.get_format_column(i) for i in omics]
    # formated_data

    method = "lmg"

    relimp_df_Ummae_full = (
        calc_relimp(
            formated_data,
            label="Stenosis",
            vars=omics_format,
            method=method,
            binary_y=True,
        )
        .assign(
            Method=method,
        )
        .melt(
            id_vars=["Method", "R2.decomp"],
            value_vars=omics_format,
            var_name="Omics",
            value_name="Importance",
        )
    )

    # 2) cal relative importance for Real values
    omics = all_clinical_ummae_traits + basic

    RelativeImportanceData = train_data[omics + ["Stenosis"]].copy()

    RelativeImportanceDataFromat = columnsFormatV1(RelativeImportanceData)
    formated_data = RelativeImportanceDataFromat.format(RelativeImportanceData)

    omics_format = [RelativeImportanceDataFromat.get_format_column(i) for i in omics]
    # formated_data

    method = "lmg"

    relimp_df_Real = (
        calc_relimp(
            formated_data,
            label="Stenosis",
            vars=omics_format,
            method=method,
            binary_y=True,
        )
        .assign(
            Method=method,
        )
        .melt(
            id_vars=["Method", "R2.decomp"],
            value_vars=omics_format,
            var_name="Omics",
            value_name="Importance",
        )
    )

    # 3) merged data
    RelativeImportanceData = (
        relimp_df_Real[["Omics", "Importance"]]
        .merge(
            relimp_df_Ummae_full[["Omics", "Importance"]].assign(
                Omics=relimp_df_Ummae_full["Omics"].str.replace("_Ummae", "")
            ),
            suffixes=("_Real", "_Predicted"),
            on="Omics",
        )
        .sort_values("Importance_Real", ascending=False)
    )
    RelativeImportanceData["Importance_Predicted_plt"] = -RelativeImportanceData[
        "Importance_Predicted"
    ]
    RelativeImportanceData["Omics"] = RelativeImportanceData["Omics"].replace(
        {
            "HbA1C_": "HbA1C",
            "LPa": "LP(a)",
            "hs_CRP": "hs-CRP",
            "D_Dimer": "D-Dimer",
            "CK_MB": "CK-MB",
        }
    )
    RelativeImportanceData.to_csv(RelativeDataDir, index=False)
else:
    RelativeImportanceData = pd.read_csv(RelativeDataDir)

RelativeImportanceData

,Omics,Importance_Real,Importance_Predicted,Importance_Predicted_plt
0,HDLC,0.231729,0.303351,-0.303351
1,HbA1C,0.169712,0.169296,-0.169296
2,Sex,0.131286,0.115455,-0.115455
3,BNP,0.094023,0.006040,-0.006040
4,TC,0.067828,0.041242,-0.041242
5,hs-CRP,0.057281,0.005575,-0.005575
6,nonHDL,0.046326,0.018002,-0.018002
7,LP(a),0.040840,0.022085,-0.022085
8,Height,0.034138,0.023628,-0.023628
9,hsTnI,0.022960,0.003599,-0.003599
